In [1]:
import torch
import torch.nn as nn

from torch_frft.dfrft_module import dfrft
from torch_frft.frft_module import frft
from torch_frft.layer import DFrFTLayer, FrFTLayer

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def train(model, optimizer, data, target, epochs=1000):
    global a_original
    for epoch in range(epochs):
        optimizer.zero_grad()
        loss = torch.norm(target - model(data))
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")
    print("Original  a:", a_original)
    print("Estimated a:", model[0].order)

In [4]:
num_samples, seq_length = 100, 16
a_original = torch.tensor(1.45, dtype=torch.float32)
a_initial = 1.35
lr = 1e-2
epochs = 500
X = torch.randn(num_samples, seq_length, dtype=torch.float32, device=device)

In [5]:
Y = frft(X, a_original)
model = nn.Sequential(FrFTLayer(order=a_initial)).to(device)
train(
    model=model,
    optimizer=torch.optim.Adagrad(model.parameters(), lr=lr),
    data=X,
    target=Y,
    epochs=epochs,
)

Epoch [0/500], Loss: 44.8892
Epoch [100/500], Loss: 0.0000
Epoch [200/500], Loss: 0.0000
Epoch [300/500], Loss: 0.0000
Epoch [400/500], Loss: 0.0000
Original  a: tensor(1.4500)
Estimated a: Parameter containing:
tensor(1.4500, device='cuda:0', requires_grad=True)


In [6]:
Y = dfrft(X, a_original)
model = nn.Sequential(DFrFTLayer(order=a_initial)).to(device)
train(
    model=model,
    optimizer=torch.optim.Adam(model.parameters(), lr=lr),
    data=X,
    target=Y,
    epochs=epochs,
)

Epoch [0/500], Loss: 48.9082
Epoch [100/500], Loss: 1.3701
Epoch [200/500], Loss: 1.4373
Epoch [300/500], Loss: 0.7099
Epoch [400/500], Loss: 0.5322
Original  a: tensor(1.4500)
Estimated a: Parameter containing:
tensor(1.4514, device='cuda:0', requires_grad=True)
